In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm
import scipy.linalg
import seaborn as sns

In [9]:
df_dict = pd.read_excel('/Users/stilo/Downloads/portfolio.xlsx', sheet_name=None)


In [10]:
df = pd.DataFrame.from_dict(df_dict['Sheet1'])

In [12]:
df.head(10)

,CounterPartyID,EAD,PD,Sector,Region
0,1,211355,0.024910,S1,R5
1,2,792035,0.027178,S3,R2
2,3,82965,0.032871,S4,R1
3,4,453682,0.013627,S6,R5
4,5,404525,0.006988,S4,R4
5,6,1399295,0.032677,S4,R5
6,7,395168,0.049545,S4,R3
7,8,135192,0.005525,S5,R4
8,9,867514,0.047397,S1,R3
9,10,986297,0.045538,S1,R2


In [13]:
df['Z_def'] = norm.ppf(df['PD'])
df.head()

,CounterPartyID,EAD,PD,Sector,Region,Z_def
0,1,211355,0.024910,S1,R5,-1.961503
1,2,792035,0.027178,S3,R2,-1.923995
2,3,82965,0.032871,S4,R1,-1.840185
3,4,453682,0.013627,S6,R5,-2.207873
4,5,404525,0.006988,S4,R4,-2.457860


In [33]:
var_no_conc = pd.DataFrame(index=range(len(df)), columns= range(num_simulations))
alpha = 0.2  # Some value for alpha
LGD = 0.15
num_simulations = 500  # Number of Monte Carlo simulations
num_assets = len(df)

def var1(df):

    var_no_conc = pd.DataFrame(index=range(num_assets), columns= range(num_simulations))

    for i in range(num_simulations):

        Y = np.random.normal(size=(1,))

        for j in range(num_assets):
        
            I = norm.cdf((df['Z_def'][j] - np.sqrt(alpha) * Y) / np.sqrt(1 - alpha)) * df['EAD'][j] * LGD
            var_no_conc.iloc[j,i] = I 

    tot_var1_k = var_no_conc.sum()
    var1_final = tot_var1_k.mean()
    

    return var_no_conc, tot_var1_k, var1_final


In [ ]:
var_no_conc = pd.DataFrame(index=range(len(df)), columns= range(num_simulations))
alpha = 0.2  # Some value for alpha
LGD = 0.15
num_assets = len(df)

In [48]:
num_simulations = 5000
loss_dist = []
for i in range(num_simulations):

    Y = np.random.normal()
    total_loss = 0
    for j in range(num_assets):
        I = norm.cdf((df['Z_def'][j] - np.sqrt(alpha) * Y) / np.sqrt(1 - alpha)) * df['EAD'][j] * LGD

        total_loss += I
    loss_dist.append(total_loss)

In [49]:
VaR_99 = np.percentile(loss_dist, 99.9)
VaR_95 = np.percentile(loss_dist, 95)
VaR_90 = np.percentile(loss_dist, 90)
print(VaR_99)
print(VaR_95)
print(VaR_90)


13261095.205638412
5304516.170199957
3847693.694419971


## Question 2

In [53]:
var_no_conc = pd.DataFrame(index=range(len(df)), columns= range(num_simulations))
alpha = 0.2  # Some value for alpha
LGD = 0.15
beta_s = 0.06
num_assets = len(df)

In [54]:
num_simulations = 100
loss_dist = []

for i in range(num_simulations):

    Y = np.random.normal()
    Z_s = {}  # Dictionary to store Z_s for each sector
    for sector in df['Sector'].unique():
        Z_s[sector] = np.random.normal()    
    
    total_loss = 0
    for j in range(num_assets):
        sector = df.loc[j, 'Sector']

        I = norm.cdf((df['Z_def'][j] - np.sqrt(alpha) * Y - np.sqrt(beta_s) * Z_s[sector]) / np.sqrt(1 - alpha - beta_s)) * df['EAD'][j] * LGD

        total_loss += I
    loss_dist.append(total_loss)

In [55]:
VaR_99 = np.percentile(loss_dist, 99.9)
VaR_95 = np.percentile(loss_dist, 95)
VaR_90 = np.percentile(loss_dist, 90)
print(VaR_99)
print(VaR_95)
print(VaR_90)

7817341.171458791
4908378.470900172
4352224.895705567


## QUESTION 3

In [57]:
var_no_conc = pd.DataFrame(index=range(len(df)), columns= range(num_simulations))
alpha = 0.2  # Some value for alpha
LGD = 0.15
beta_r = 0.04
num_assets = len(df)

In [58]:
num_simulations = 100
loss_dist = []

for i in range(num_simulations):

    Y = np.random.normal()
    Z_r = {}  # Dictionary to store Z_s for each sector
    for region in df['Region'].unique():
        Z_r[region] = np.random.normal()    
    
    total_loss = 0
    for j in range(num_assets):
        region = df.loc[j, 'Region']

        I = norm.cdf((df['Z_def'][j] - np.sqrt(alpha) * Y - np.sqrt(beta_r) * Z_r[region]) / np.sqrt(1 - alpha - beta_r)) * df['EAD'][j] * LGD

        total_loss += I
    loss_dist.append(total_loss)

In [59]:
VaR_99 = np.percentile(loss_dist, 99.9)
VaR_95 = np.percentile(loss_dist, 95)
VaR_90 = np.percentile(loss_dist, 90)
print(VaR_99)
print(VaR_95)
print(VaR_90)

7442227.786964865
5203525.352025925
3746915.9932441413


## QUESTION 4

In [60]:
var_no_conc = pd.DataFrame(index=range(len(df)), columns= range(num_simulations))
alpha = 0.2  # Some value for alpha
LGD = 0.15
beta_r = 0.04
num_assets = len(df)

In [63]:
num_simulations = 100
loss_dist = []

for i in range(num_simulations):

    Y = np.random.normal()
    
    total_loss = 0
    for j in range(num_assets):

        Phi =  np.random.normal()
        X = np.sqrt(alpha) * Y + np.sqrt(1-alpha)*Phi 

        if X <= df['Z_def'][j]:
            I = df['EAD'][j] * LGD

        else:
            I = 0
        
        total_loss += I
    loss_dist.append(total_loss)

In [64]:
VaR_99 = np.percentile(loss_dist, 99.9)
VaR_95 = np.percentile(loss_dist, 95)
VaR_90 = np.percentile(loss_dist, 90)
print(VaR_99)
print(VaR_95)
print(VaR_90)

10277313.781650012
3798961.882499998
2497301.5800000015


## QUESTION 5

In [65]:
num_simulations = 100
loss_dist = []

for i in range(num_simulations):

    Y = np.random.normal()
    
    total_loss = 0

    Z_r = {}  # Dictionary to store Z_s for each sector
    for region in df['Region'].unique():
        Z_r[region] = np.random.normal() 

    Z_s = {}  # Dictionary to store Z_s for each sector
    for sector in df['Sector'].unique():
        Z_s[sector] = np.random.normal() 
    
    for j in range(num_assets):

        Phi =  np.random.normal()
        region = df.loc[j, 'Region']
        sector = df.loc[j, 'Sector']

        X = np.sqrt(alpha) * Y + np.sqrt(beta_r)*Z_r[region] + np.sqrt(beta_s) * Z_s[sector] + np.sqrt(1-alpha-beta_r-beta_s)*Phi

        if X <= df['Z_def'][j]:
            I = df['EAD'][j] * LGD
        else:
            I = 0
        
        total_loss += I
    loss_dist.append(total_loss)

In [66]:
VaR_99 = np.percentile(loss_dist, 99.9)
VaR_95 = np.percentile(loss_dist, 95)
VaR_90 = np.percentile(loss_dist, 90)
print(VaR_99)
print(VaR_95)
print(VaR_90)

14145851.35530001
5571974.752499991
3562151.685000001


In [68]:
sqrt_alpha = np.sqrt(alpha)

num_simulations = 500
var1_dist = []
var2_dist = []
var3_dist = []
var4_dist = []
var5_dist = []

for i in range(num_simulations):

    Y = np.random.normal()
    Y_alpha = sqrt_alpha * Y
    
    var1 = 0
    var2 = 0
    var3 = 0
    var4 = 0
    var5 = 0

    Z_r = {}  # Dictionary to store Z_s for each sector
    for region in df['Region'].unique():
        Z_r[region] = np.random.normal() 

    Z_s = {}  # Dictionary to store Z_s for each sector
    for sector in df['Sector'].unique():
        Z_s[sector] = np.random.normal() 
    
    for j in range(num_assets):

        # VaR1
        I1 = norm.cdf((df['Z_def'][j] - np.sqrt(alpha) *Y) / np.sqrt(1 - alpha)) * df['EAD'][j] * LGD

        # VaR2
        I2 = norm.cdf((df['Z_def'][j] - np.sqrt(alpha) * Y - np.sqrt(beta_s) * Z_s[sector]) / np.sqrt(1 - alpha - beta_s)) * df['EAD'][j] * LGD

        # Var3
                
        I3 = norm.cdf((df['Z_def'][j] - np.sqrt(alpha) * Y - np.sqrt(beta_r) * Z_r[region]) / np.sqrt(1 - alpha - beta_r)) * df['EAD'][j] * LGD

        # Var4 
        Phi =  np.random.normal()
        X4 = np.sqrt(alpha) * Y + np.sqrt(1-alpha)*Phi 

        if X4 <= df['Z_def'][j]:
            I4 = df['EAD'][j] * LGD
        else:
            I4 = 0

        # Var5
        X5 = np.sqrt(alpha) * Y + np.sqrt(beta_r)*Z_r[region] + np.sqrt(beta_s) * Z_s[sector] + np.sqrt(1-alpha-beta_r-beta_s)*Phi

        if X5 <= df['Z_def'][j]:
            I5 = df['EAD'][j] * LGD
        else:
            I5 = 0
        
        var1 += I1
        var2 += I2
        var3 += I3
        var4 += I4
        var5 += I5

    var1_dist.append(var1)
    var2_dist.append(var2)
    var3_dist.append(var3)
    var4_dist.append(var4)
    var5_dist.append(var5)

In [ ]:
VaR_99 = np.percentile(loss_dist, 99.9)
VaR_95 = np.percentile(loss_dist, 95)
VaR_90 = np.percentile(loss_dist, 90)
print(VaR_99)
print(VaR_95)
print(VaR_90)

## Final optimized code

In [76]:
alpha = 0.2
LGD = 0.15
beta_r = 0.04
beta_s = 0.06

In [73]:
np.random.seed(123)

num_simulations = 1000000
num_assets = len(df)

# Convert DataFrame columns to numpy arrays for vectorized operations
Z_def = df['Z_def'].values
EAD = df['EAD'].values

# Initialize results arrays
var1_dist = np.zeros(num_simulations)
var2_dist = np.zeros(num_simulations)
var3_dist = np.zeros(num_simulations)
var4_dist = np.zeros(num_simulations)
var5_dist = np.zeros(num_simulations)

# Pre-compute the square roots since they are constants
sqrt_alpha = np.sqrt(alpha)
sqrt_1_minus_alpha = np.sqrt(1 - alpha)
sqrt_beta_s = np.sqrt(beta_s)
sqrt_beta_r = np.sqrt(beta_r)

# Create a dictionary to map each sector and region to an index
sector_indices = {sector: i for i, sector in enumerate(df['Sector'].unique())}
region_indices = {region: i for i, region in enumerate(df['Region'].unique())}

# Pre-calculate Z_s and Z_r for each simulation
Z_s_matrix = np.random.normal(size=(num_simulations, len(sector_indices)))
Z_r_matrix = np.random.normal(size=(num_simulations, len(region_indices)))

for i in range(num_simulations):
    Y = np.random.normal()

    # Get the Z values for the current simulation
    Z_s_current = Z_s_matrix[i, df['Sector'].map(sector_indices).values]
    Z_r_current = Z_r_matrix[i, df['Region'].map(region_indices).values]

    var1_dist[i] = np.sum(norm.cdf((Z_def - sqrt_alpha * Y) / sqrt_1_minus_alpha) * EAD * LGD)

    var2_dist[i] = np.sum(norm.cdf((Z_def - sqrt_alpha * Y - sqrt_beta_s * Z_s_current) / np.sqrt(1 - alpha - beta_s)) * EAD * LGD)

    var3_dist[i] = np.sum(norm.cdf((Z_def - sqrt_alpha * Y - sqrt_beta_r * Z_r_current) / np.sqrt(1 - alpha - beta_r)) * EAD * LGD)

    Phi = np.random.normal(size=num_assets)
    X4 = sqrt_alpha * Y + sqrt_1_minus_alpha * Phi
    var4_dist[i] = np.sum((X4 <= Z_def) * EAD * LGD)

    X5 = sqrt_alpha * Y + sqrt_beta_r * Z_r_current + sqrt_beta_s * Z_s_current + np.sqrt(1 - alpha - beta_r - beta_s) * Phi
    var5_dist[i] = np.sum((X5 <= Z_def) * EAD * LGD)

var1_dist = np.array(var1_dist)
var2_dist = np.array(var2_dist)
var3_dist = np.array(var3_dist)
var4_dist = np.array(var4_dist)
var5_dist = np.array(var5_dist)

In [75]:
var_arrays = [var1_dist, var2_dist, var3_dist, var4_dist, var5_dist]
percentiles = [99.9, 95, 90]

for i, var_dist in enumerate(var_arrays):
    print(f"VaR {i+1}:")
    for percentile in percentiles:
        VaR = np.percentile(var_dist, percentile)
        print(f"  VaR at {percentile}th percentile: {VaR}")

VaR 1:
  VaR at 99.9th percentile: 14100104.176809002
  VaR at 95th percentile: 5004397.22024683
  VaR at 90th percentile: 3630198.9357116427
VaR 2:
  VaR at 99.9th percentile: 14669617.02125226
  VaR at 95th percentile: 5139547.938746887
  VaR at 90th percentile: 3690954.847571018
VaR 3:
  VaR at 99.9th percentile: 14725673.961616915
  VaR at 95th percentile: 5126915.195501309
  VaR at 90th percentile: 3684186.0718265506
VaR 4:
  VaR at 99.9th percentile: 17843223.667050548
  VaR at 95th percentile: 8909806.282499993
  VaR at 90th percentile: 3376307.1149999998
VaR 5:
  VaR at 99.9th percentile: 18466697.270250224
  VaR at 95th percentile: 8964027.674999995
  VaR at 90th percentile: 3471290.7299999986
